In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import gc
import random
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader


In [ ]:
#HDKIM
MAX_SEQ = 100
#HDKIMHDKIM

## Load data

In [ ]:
%%time
dtype = {'timestamp': 'int64', 
         'user_id': 'int64' ,
         'content_id': 'int16',
         'content_type_id': 'int8',
         'answered_correctly':'int8',
         'prior_question_elapsed_time':'float64'}

train_df = pd.read_csv('/kaggle/input/saktmodel/train_200_valid_2.csv', usecols=[1,2,3,4,7,8], dtype=dtype)
train_df.head()


In [ ]:
train_df = train_df[train_df.content_type_id == False]

#arrange by timestamp
train_df = train_df.sort_values(['timestamp'], ascending=True).reset_index(drop = True)

In [ ]:
valid = pd.read_csv('/kaggle/input/saktmodel/valid_2.csv', usecols=[ 2, 3,4,5,8,9], dtype=dtype)
valid = valid[valid.content_type_id == False]

valid.head(10)

In [ ]:
question_com = pd.read_csv('/kaggle/input/saktques/question_cmnts.csv')
question_com.columns = ['question_id', 'community']

In [ ]:
# question_com

In [ ]:
# question_com

In [ ]:
question_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')
question_df['part'].fillna(8, inplace=True)
question_df['part'] = question_df['part']-1

In [ ]:
question_df.head(10)

In [ ]:
question_df = pd.merge(question_df, question_com, on='question_id', how='left')

In [ ]:
question_df.loc[question_df['community']==4]

In [ ]:
del question_com
gc.collect()


In [ ]:
question_df

In [ ]:
part_skill = question_df['part'].unique()
print("part is:",part_skill)
n_part_skill = len(part_skill)
print("number part skills", n_part_skill)

In [ ]:
com_skill = question_df['community'].unique()
print("community is:", com_skill)
n_com_skill = len(com_skill)
print("number community skills is:", n_com_skill)

In [ ]:
elapsed_mean = train_df.prior_question_elapsed_time.mean()
elapsed_mean

In [ ]:
train_df['prior_question_elapsed_time'].fillna(elapsed_mean, inplace=True)


In [ ]:
def get_elapsed_time(ela):
    ela = ela // 1000
    if ela > 300:
        return 300
    else:
        return ela

In [ ]:
train_df['prior_question_elapsed_time'] = train_df['prior_question_elapsed_time'].apply(lambda x: get_elapsed_time(x))

In [ ]:
train_df = pd.merge(train_df, question_df, left_on='content_id',right_on='question_id',how='left')
valid = pd.merge(valid, question_df, left_on='content_id',right_on='question_id',how='left')

## Preprocess

In [ ]:
# skills = train_df["content_id"].unique()
n_skill = 13523
print("number skills", n_skill)

In [ ]:
ela_skill = train_df['prior_question_elapsed_time'].unique()
n_ela_skill = 301
print("number ela skills", n_ela_skill)
print("ela is:", ela_skill)

In [ ]:
valid['prior_question_elapsed_time'].fillna(elapsed_mean, inplace=True)
valid['prior_question_elapsed_time'] = valid['prior_question_elapsed_time'].apply(lambda x: get_elapsed_time(x))

In [ ]:
group = train_df[['user_id', 'part','content_id','community', 'answered_correctly','prior_question_elapsed_time']].groupby('user_id').apply(lambda r: (
            r['content_id'].values,
            r['answered_correctly'].values,
            r['prior_question_elapsed_time'].values,
            r['part'].values,
            r['community'].values))



del train_df
gc.collect()

In [ ]:
#HDKIM
import random
random.seed(1)
#HDKIMHDKIM

In [ ]:
class SAKTDataset(Dataset):
    def __init__(self, group, n_skill,n_ela_skill, max_seq=MAX_SEQ): #HDKIM 100
        super(SAKTDataset, self).__init__()
        self.max_seq = max_seq
        self.n_skill = n_skill
        self.n_ela_skill = n_ela_skill
        self.samples = group
        
#         self.user_ids = [x for x in group.index]
        self.user_ids = []
        for user_id in group.index:
            q, qa,ela,part,com = group[user_id]
            if len(q) < 2: #HDKIM 10
                continue
            self.user_ids.append(user_id)
            
            #HDKIM Memory reduction
#             if len(q)>self.max_seq:
#                 group[user_id] = (q[-self.max_seq:],qa[-self.max_seq:], ela[-self.max_seq:])

    def __len__(self):
        return len(self.user_ids)

    def __getitem__(self, index):
        user_id = self.user_ids[index]
        q_, qa_,ela_,part_,com_ = self.samples[user_id]
        seq_len = len(q_)

        q = np.ones(self.max_seq, dtype=int)*13523
        qa = np.zeros(self.max_seq, dtype=int)
        ela = np.ones(self.max_seq, dtype=int)*301
        part = np.ones(self.max_seq, dtype=int)*7
        com = np.ones(self.max_seq, dtype=int)*4

        if seq_len >= self.max_seq:
            if random.random() >0.1:
                start = random.randint(0, (seq_len-self.max_seq))
                end = start + self.max_seq
                q[:] = q_[start:end]
                qa[:] = qa_[start:end]
                ela[:] = ela_[start:end]
                part[:] = part_[start:end]
                com[:] = com_[start:end]

            else:   
                q[:] = q_[-self.max_seq:]
                qa[:] = qa_[-self.max_seq:]
                ela[:] = ela_[-self.max_seq:]
                part[:] = part_[-self.max_seq:]
                com[:] = com_[-self.max_seq:]
                
                
        else:
            if random.random() > 0.1:
                start = 0
                end = random.randint(2, seq_len)
                seq_len = end -start
                q[-seq_len:] = q_[0:seq_len]
                qa[-seq_len:] = qa_[0:seq_len]
                ela[-seq_len:] = ela_[0:seq_len]
                part[-seq_len:] = part_[0:seq_len]
                com[-seq_len:] = com_[0:seq_len]
                
            else:
                q[-seq_len:] = q_
                qa[-seq_len:] = qa_
                ela[-seq_len:] = ela_   
                part[-seq_len:] = part_   
                com[-seq_len:] = com_   
            
        label = qa[:]
        
        # encoder输入，从0开始一直到len 
        enc_que = np.ones(self.max_seq, dtype=int)*13523
        enc_que = q[:].copy()
        
        enc_part = np.ones(self.max_seq, dtype=int)*7
        enc_part = part[:].copy()
        
        enc_com = np.ones(self.max_seq, dtype=int)*4
        enc_com = com[:].copy()
        
        # decoder输入， 先在左边pad一个位置，然后从0一直到len-1
        dec_ela = np.ones(self.max_seq, dtype=int)*301
        dec_ela[1:] = ela[:-1].copy()
        
        dec_ans = np.zeros(self.max_seq, dtype=int)
        dec_ans[1:] = qa[:-1].copy()
        

        
        return enc_que, enc_part, enc_com, dec_ans,  label
        
        
#         return x, target_id,ela_x,ela_target,part_x,part_target,com_x,com_target,ans_x,label
    # x和target_id都是来自于q，存在一个错位，x比target_id提前一个位置
    # 所以我们可以尝试将ela 也分为ela_x和ela_target_id

In [ ]:
dataset = SAKTDataset(group, n_skill, n_ela_skill)
dataloader = DataLoader(dataset, batch_size=2048, shuffle=True,num_workers=8)

item = dataset.__getitem__(5)


In [ ]:
class ValidDataset(Dataset):
    def __init__(self, samples, test_df, n_skill,n_ela_skill, max_seq=MAX_SEQ): #HDKIM 100
        super(ValidDataset, self).__init__()
        self.samples = samples
        self.user_ids = [x for x in test_df["user_id"].unique()]
        self.test_df = test_df
        self.n_ela_skill = n_ela_skill
        self.n_skill = n_skill
        self.max_seq = max_seq

    def __len__(self):
        return self.test_df.shape[0]

    def __getitem__(self, index):
        test_info = self.test_df.iloc[index]

        user_id = test_info["user_id"]
        enc_target_que = test_info["content_id"]
#         dec_target_ela = test_info["prior_question_elapsed_time"]
        enc_target_part = test_info["part"]
        enc_target_com = test_info["community"]

        
        label = test_info['answered_correctly']

        q = np.ones(self.max_seq, dtype=int)*13523
        qa = np.zeros(self.max_seq, dtype=int)
        ela = np.ones(self.max_seq, dtype=int)*301
        part = np.ones(self.max_seq, dtype=int)*7
        com = np.ones(self.max_seq, dtype=int)*4

        if user_id in self.samples.index:
            q_, qa_, ela_, part_,com_ = self.samples[user_id]
            
            seq_len = len(q_)

            if seq_len >= self.max_seq:
                q = q_[-self.max_seq:]
                qa = qa_[-self.max_seq:]
                ela = ela_[-self.max_seq:]
                part = part_[-self.max_seq:]
                com = com_[-self.max_seq:]
                
            else:
                q[-seq_len:] = q_
                qa[-seq_len:] = qa_       
                ela[-seq_len:] = ela_
                part[-seq_len:] = part_
                com[-seq_len:] = com_
        
        
        # encoder的输入
        enc_que = np.ones(self.max_seq, dtype=int)*13523
        enc_que = np.append(q[1:].copy(),[enc_target_que])
        
        enc_part = np.ones(self.max_seq, dtype=int)*7
        enc_part = np.append(part[1:].copy(), [enc_target_part])
        
        enc_com = np.ones(self.max_seq, dtype=int)*4
        enc_com = np.append(com[1:].copy(), [enc_target_com])
        
        # decoder的输入
        dec_ans = np.zeros(self.max_seq, dtype=int)
        dec_ans = qa[:].copy()
        


        return enc_que, enc_part, enc_com, dec_ans, np.array([label])

In [ ]:
test_dataset = ValidDataset(group, valid, n_skill, n_ela_skill)
test_dataloader = DataLoader(test_dataset, batch_size=2048, shuffle=False,num_workers=8)

## Define model

In [ ]:
def future_mask(seq_length):
    future_mask = np.triu(np.ones((seq_length, seq_length)), k=1).astype('bool')
    return torch.from_numpy(future_mask)

In [ ]:
class FFN(nn.Module):
    def __init__(self, state_size=200):
        super(FFN, self).__init__()
        self.state_size = state_size

        self.lr1 = nn.Linear(state_size, state_size)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(state_size, state_size)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        return self.dropout(x)

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, device="cpu"):
        super().__init__( )
        
        self.device = device
        self.multi_attn = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads)
        self.ffn = FFN(embed_dim)
        self.layer_norm1 = nn.LayerNorm(embed_dim)
        self.layer_norm2 = nn.LayerNorm(embed_dim)
        
    def forward(self, enc_que, enc_part, enc_com, enc_pos):
        out = enc_que + enc_part + enc_com + enc_pos
        
        out = out.permute(1,0,2)
        
        # Multihead attention
        n,_,_ = out.shape
        out = self.layer_norm1(out)
        skip_out = out
        out, attn = self.multi_attn(out, out, out, 
                                    attn_mask=future_mask(seq_length=n).to(self.device))
            
        out = out + skip_out
        
        # FeedForward
        out = out.permute(1,0,2)
        out = self.layer_norm2(out)
        skip_out = out
        out = self.ffn(out)
        out = out + skip_out
        
        return out
        

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, device="cpu"):
        super().__init__()
        
        self.device = device
        self.multi_attn1 = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads)
        self.multi_attn2 = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads)
        self.ffn = FFN(embed_dim)
        
        self.layer_norm1 = nn.LayerNorm(embed_dim)
        self.layer_norm2 = nn.LayerNorm(embed_dim)
        self.layer_norm3 = nn.LayerNorm(embed_dim)
        
    def forward(self, enc_out, dec_ans, dec_pos):
        out = dec_ans + dec_pos
        
        out = out.permute(1,0,2)
        n,_,_ = out.shape
        
        # Decoder slef Multihead Attention 
        out = self.layer_norm1(out)
        skip_out = out
        out, attn = self.multi_attn1(out, out, out,
                                     attn_mask=future_mask(seq_length=n).to(self.device))
        
        out = out + skip_out
        
        # Encoder-Decoder Multihead Attention
        enc_out = enc_out.permute(1,0,2)
#         enc_out = self.layer_norm2(enc_out)
        enc_out = self.layer_norm2(enc_out)
        
        skip_out = out
        
        out, attn = self.multi_attn2(out, enc_out, enc_out,
                                     attn_mask=future_mask(seq_length=n).to(self.device))
        out = out + skip_out
        
        
        # FeedForward 
        out = out.permute(1,0,2)
#         out = self.layer_norm3(out)
        out = self.layer_norm3(out)
        
        skip_out = out
        out = self.ffn(out)
        out = out + skip_out
        
        return out
    

In [ ]:
class SaintModel(nn.Module):
    def __init__(self,num_que, num_part, num_com, num_ans=2, num_heads=8, 
                 max_seq=MAX_SEQ, embed_dim=128, device = "cpu"):
        super(SaintModel, self).__init__()
        
        self.device = device
        self.num_que = num_que
        self.num_part = num_part
        self.num_com = num_com
        self.num_ans = num_ans
        self.embed_dim = embed_dim
        
        self.que_embedding = nn.Embedding(num_que, embed_dim)
        self.part_embedding = nn.Embedding(num_part, embed_dim)
        self.com_embedding = nn.Embedding(num_com, embed_dim)
        self.ans_embedding = nn.Embedding(num_ans, embed_dim)
        self.pos_embedding = nn.Embedding(max_seq+1, embed_dim)
        
        self.encoder = EncoderBlock(embed_dim, num_heads, device)
        self.decoder = DecoderBlock(embed_dim, num_heads, device)
        
        self.pred = nn.Linear(embed_dim, 1)
    
    def forward(self, enc_que, enc_part, enc_com, dec_ans):
#         device = enc_que.device
        
        enc_que = self.que_embedding(enc_que)
        enc_part = self.part_embedding(enc_part)
        enc_com = self.com_embedding(enc_com)
        dec_ans = self.ans_embedding(dec_ans)
        
        enc_pos = torch.arange(1,1+enc_que.size(1)).unsqueeze(0).to(self.device)
        dec_pos = torch.arange(enc_que.size(1)).unsqueeze(0).to(self.device)
        
        enc_pos = self.pos_embedding(enc_pos)
        dec_pos = self.pos_embedding(dec_pos)
        
        
        enc_out = self.encoder(enc_que, enc_part, enc_com, enc_pos)
        dec_out = self.decoder(enc_out, dec_ans, dec_pos)
        
        out = self.pred(dec_out)
        
        return out.squeeze(-1)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = xm.xla_device()

model = SaintModel(n_skill+1,n_part_skill+1,n_com_skill, embed_dim=128, device=device)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.99, weight_decay=0.005)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

model.to(device)
criterion.to(device)

In [ ]:
def train_epoch(model, train_iterator, optim, criterion, device="cpu"):
    model.train()

    train_loss = []
    num_corrects = 0
    num_total = 0
    labels = []
    outs = []

    tbar = tqdm(train_iterator)
    for item in tbar:
        enc_que = item[0].to(device).long()
        enc_part = item[1].to(device).long()
        enc_com = item[2].to(device).long()
        dec_ans = item[3].to(device).long()
        
        label = item[4].to(device).float()        
        
        optim.zero_grad()
        output = model(enc_que, enc_part, enc_com, dec_ans)
        loss = criterion(output, label)
        loss.backward()
        optim.step()
#         xm.mark_step()
        train_loss.append(loss.item())

        output = output[:, -1]
        label = label[:, -1] 
        pred = (torch.sigmoid(output) >= 0.5).long()
        
        num_corrects += (pred == label).sum().item()
        num_total += len(label)

        labels.extend(label.view(-1).data.cpu().numpy())
        outs.extend(output.view(-1).data.cpu().numpy())

        tbar.set_description('loss - {:.4f}'.format(loss))

    acc = num_corrects / num_total
    auc = roc_auc_score(labels, outs)
    loss = np.mean(train_loss)

    return loss, acc, auc

In [ ]:
def evaluate_epoch(model, test_dataloader, criterion, device="cpu"):
    model.eval()
    eval_loss = []
    num_corrects = 0
    num_total = 0
    labels = []
    outs = []
    
    with torch.no_grad():
        for item in tqdm(test_dataloader):
            enc_que = item[0].to(device).long()
            enc_part = item[1].to(device).long()
            enc_com = item[2].to(device).long()
            dec_ans = item[3].to(device).long()
        
            label = item[4].to(device).float()        
        
#             print("In evaluation x shape is:{}\t taret_id shape is:{}\t label shape is:{}".format(x.shape, target_id.shape,label.shape))
            
            output = model(enc_que, enc_part, enc_com, dec_ans)
#             print("Output shape is:{}\tlabel shape is:{}".format(output.shape, label.shape))
      
            output_loss = output[:, -1:]
            loss = criterion(output_loss, label)
            eval_loss.append(loss.item())
            
            output = output[:,-1]
            label = label[:, -1] 
            pred = (torch.sigmoid(output) >= 0.5).long()
            
            num_corrects += (pred == label).sum().item()
            num_total += len(label)

            labels.extend(label.view(-1).data.cpu().numpy())
            outs.extend(output.view(-1).data.cpu().numpy())

    acc = num_corrects / num_total
#     print("In evaluate labels is:{}\t outs is:{}\t".format(labels, outs))

    auc = roc_auc_score(labels, outs)
#     auc = 0

    loss = np.mean(eval_loss)   
    
    return loss, acc, auc

In [ ]:
epochs = 40
best_auc = 0.0

for epoch in range(epochs):
    loss, acc, auc = train_epoch(model, dataloader, optimizer, criterion, device)
    print("epoch - {} train_loss - {:.2f} acc - {:.3f} auc - {:.3f}".format(epoch, loss, acc, auc))
    
    if epoch %10 == 0 or epoch == 79:
        eval_loss, eval_acc, eval_auc = evaluate_epoch(model, test_dataloader, criterion, device)
        print("epoch - {} eval_loss - {:.2f} eval acc - {:.3f} eval auc - {:.3f}".format(epoch, eval_loss, eval_acc, eval_auc))
        if best_auc < eval_auc: 
            best_auc = eval_auc
            torch.save(model.state_dict(),'best_model.pt')




In [ ]:
import gc
del dataset
gc.collect()

## Test

In [ ]:
class TestDataset(Dataset):
    def __init__(self, samples, test_df, n_skill, n_ela_skill,n_part_skill,max_seq=MAX_SEQ): #HDKIM 100
        super(TestDataset, self).__init__()
        self.samples = samples
        self.user_ids = [x for x in test_df["user_id"].unique()]
        self.test_df = test_df
        self.n_ela_skill = n_ela_skill
        self.n_part_skill = n_part_skill
        self.n_skill = n_skill
        self.max_seq = max_seq

    def __len__(self):
        return self.test_df.shape[0]

    def __getitem__(self, index):
        test_info = self.test_df.iloc[index]

        user_id = test_info["user_id"]
        target_id = test_info["content_id"]
        ela_target_id = test_info["prior_question_elapsed_time"]
        part_target_id = test_info["part"]
        com_target_id = test_info["community"]

        q = np.ones(self.max_seq, dtype=int)*13523
        qa = np.zeros(self.max_seq, dtype=int)
        ela = np.ones(self.max_seq, dtype=int)*301
        part = np.ones(self.max_seq, dtype=int)*7
        com = np.ones(self.max_seq, dtype=int)*4

        if user_id in self.samples.index:
#             print("self.samples[user_id] is:{}\n length is:{}".format(
#                 self.samples[user_id], len(self.samples[user_id])))
            q_, qa_, ela_,part_,com_ = self.samples[user_id]
            seq_len = len(q_)

            if seq_len >= self.max_seq:
                q = q_[-self.max_seq:]
                qa = qa_[-self.max_seq:]
                ela = ela_[-self.max_seq:]
                part = part_[-self.max_seq:]
                com = com_[-self.max_seq:]

            else:
                q[-seq_len:] = q_
                qa[-seq_len:] = qa_          
                ela[-seq_len:] = ela_
                part[-seq_len:] = part_
                com[-seq_len:] = com_

                
        x = np.ones(self.max_seq-1, dtype=int)*13523
        x = q[1:].copy()
#         x += (qa[1:] == 1) * self.n_skill
        
        questions = np.append(q[2:], [target_id])
        
        ela_x = np.ones(self.max_seq-1, dtype=int)*301
        ela_x = ela[1:].copy()
#         ela_x += (qa[1:] == 1) * self.n_ela_skill
        
        ela_target = np.append(ela[2:],[ela_target_id])  
        
        part_x = np.ones(self.max_seq-1, dtype=int)*7
        part_x = part[1:].copy()
        
        part_target = np.append(part[2:],[part_target_id])     
        
        com_x = np.ones(self.max_seq-1, dtype=int)*4
        com_x = com[1:].copy()
        
        com_target = np.append(com[2:],[com_target_id])        
        
        ans_x = np.zeros(self.max_seq-1, dtype=int)
        ans_x = qa[1:].copy()
        return torch.LongTensor(x), torch.LongTensor(questions),\
            torch.LongTensor(ela_x), torch.LongTensor(ela_target),\
            torch.LongTensor(part_x), torch.LongTensor(part_target),\
            torch.LongTensor(com_x), torch.LongTensor(com_target),\
            torch.LongTensor(ans_x)


In [ ]:
model = SAKTModel(n_skill+1,n_ela_skill+1,n_part_skill+1,n_com_skill,embed_dim=128)
model.load_state_dict(torch.load('best_model.pt',map_location=device))
model.to(device)

In [ ]:
import riiideducation

env = riiideducation.make_env()
iter_test = env.iter_test()
import psutil
model.eval()

#HDKIM
prev_test_df = None
#HDKIMHDKIM

for (test_df, sample_prediction_df) in tqdm(iter_test):
    #HDKIM
    if (prev_test_df is not None) & (psutil.virtual_memory().percent<90):
        print(psutil.virtual_memory().percent)
        prev_test_df['answered_correctly'] = eval(test_df['prior_group_answers_correct'].iloc[0])
        prev_test_df = prev_test_df[prev_test_df.content_type_id == False]
        prev_group = prev_test_df[['user_id', 'part','content_id','community','answered_correctly','prior_question_elapsed_time']].groupby('user_id').apply(lambda r: (
            r['content_id'].values,
            r['answered_correctly'].values,
            r['prior_question_elapsed_time'].values,
            r['part'].values,
            r['community'].values))
        for prev_user_id in prev_group.index:
            prev_group_content = prev_group[prev_user_id][0]
            prev_group_ac = prev_group[prev_user_id][1]
            prev_group_ela = prev_group[prev_user_id][2]
            prev_group_part = prev_group[prev_user_id][3]
            prev_group_com = prev_group[prev_user_id][4]

            if prev_user_id in group.index:
                group[prev_user_id] = (np.append(group[prev_user_id][0],prev_group_content), 
                                       np.append(group[prev_user_id][1],prev_group_ac),
                                       np.append(group[prev_user_id][2],prev_group_ela),
                                       np.append(group[prev_user_id][3],prev_group_part),
                                       np.append(group[prev_user_id][4],prev_group_com))
 
            else:
                group[prev_user_id] = (prev_group_content,prev_group_ac, prev_group_ela,prev_group_part,prev_group_com)
            if len(group[prev_user_id][0])>MAX_SEQ:
                new_group_content = group[prev_user_id][0][-MAX_SEQ:]
                new_group_ac = group[prev_user_id][1][-MAX_SEQ:]
                new_group_ela = group[prev_user_id][2][-MAX_SEQ:]
                new_group_part = group[prev_user_id][3][-MAX_SEQ:]
                new_group_com = group[prev_user_id][4][-MAX_SEQ:]

                group[prev_user_id] = (new_group_content,new_group_ac,new_group_ela,new_group_part,new_group_com)

    test_df['prior_question_elapsed_time'].fillna(elapsed_mean, inplace=True)
    test_df['prior_question_elapsed_time'] = test_df['prior_question_elapsed_time'].apply(lambda x: get_elapsed_time(x))
             
    test_df = pd.merge(test_df, question_df, left_on='content_id',right_on='question_id',how='left')
    prev_test_df = test_df.copy()
 
    #HDKIMHDKIM
    
    test_df = test_df[test_df.content_type_id == False]
   
    test_dataset = TestDataset(group, test_df, n_skill, n_ela_skill, n_part_skill)
    test_dataloader = DataLoader(test_dataset, batch_size=51200, 
                                 shuffle=False)
    
    outs = []

    for item in tqdm(test_dataloader):
        x = item[0].to(device).long()
        target_id = item[1].to(device).long()
        ela_x = item[2].to(device).long()
        ela_target = item[3].to(device).long()
        part_x = item[4].to(device).long()
        part_target = item[5].to(device).long()  
        com_x = item[6].to(device).long()
        com_target = item[7].to(device).long() 
        
        ans_x = item[8].to(device).long()  
        
        with torch.no_grad():
            output, att_weight = model(x, target_id,ela_x, ela_target,
                                       part_x, part_target,com_x, com_target,ans_x)
        
        
        output = torch.sigmoid(output)
        output = output[:, -1]

        # pred = (output >= 0.5).long()
        # loss = criterion(output, label)

        # val_loss.append(loss.item())
        # num_corrects += (pred == label).sum().item()
        # num_total += len(label)

        # labels.extend(label.squeeze(-1).data.cpu().numpy())
        outs.extend(output.view(-1).data.cpu().numpy())
        
    test_df['answered_correctly'] =  outs
    
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])